Angelo Salton <gsalton4@hotmail.com>

# Setup

In [2]:
import pandas as pd
import numpy as np
import joblib

from ipywidgets import interact
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Pré-processamento dos dados

In [ ]:
# salvar em formato Parquet
#X = pd.read_csv('../data/estaticos_market.csv')
#X.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# amostra
#X = X.sample(5000)

In [ ]:
# convert bool to int
#bool_cols = [*X.select_dtypes(include='bool').columns]
#X[bool_cols] = X[bool_cols].astype('int')

In [3]:
#X.to_parquet('../data/estaticos_market.parquet')
X = pd.read_parquet('../data/estaticos_market.parquet')

In [4]:
X.sample(5)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
73617,7add374a494f7fef7a96e7b22c062661523b766825c5fc...,0,EMPRESARIO INDIVIDUAL,RO,OUTROS,COMERCIO POR ATACADO,COMERCIO,1.915068,1 a 5,0,0,0,0,0,2016-11-28,0,1,0,NAO,COMERCIO POR ATACADO EXCETO 

In [5]:
X = X.sample(10000).copy()

## Features categóricas

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

In [7]:
cat_features = [*X.select_dtypes(include='object').columns]
cat_features.remove('id')

cat_imputer = SimpleImputer(strategy='most_frequent', missing_values=None)
cat_encoder = OneHotEncoder(handle_unknown='error', sparse=True, drop='if_binary')

cat_transformer = Pipeline([
    ('cat_imp', cat_imputer),
    ('cat_enc', cat_encoder)
])

## Features numéricas

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# todas as features numéricas
num_features = [*X.select_dtypes(exclude='object').columns]

In [10]:
# features numéricas: 2 estratégias para dados faltantes, de acordo com a análise exploratória
nu1_features = [i for i in num_features if str.startswith(i, 'fl_')]
nu1_imputer = SimpleImputer(strategy='most_frequent')
nu1_scaler = StandardScaler()

nu1_transformer = Pipeline([
    ('nu1_imp', nu1_imputer),
    ('nu1_sca', nu1_scaler)
])

nu2_features = [i for i in num_features if str.startswith(i, 'qt_colig')]
nu2_imputer = SimpleImputer(strategy='constant', fill_value=0)

nu2_transformer = Pipeline([
    ('nu2_imp', nu2_imputer)
])

## Consolidado

In [11]:
from sklearn.compose import ColumnTransformer

In [12]:
data_transformer = ColumnTransformer([
    ('cat_tr', cat_transformer, cat_features),
    ('nu1_tr', nu1_transformer, nu1_features),
    ('nu2_tr', nu2_transformer, nu2_features)
], remainder='drop')

In [25]:
tmp = data_transformer.fit_transform(X)
tmp.shape

(10000, 3817)

# Modelo

In [26]:
from sklearn.neighbors import NearestNeighbors

In [47]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

In [48]:
pipe = Pipeline([
    ('data_tr', data_transformer),
    ('model', knn_model)
])

In [49]:
pipe.fit(X)

Pipeline(steps=[('data_tr',
                 ColumnTransformer(transformers=[('cat_tr',
                                                  Pipeline(steps=[('cat_imp',
                                                                   SimpleImputer(missing_values=None,
                                                                                 strategy='most_frequent')),
                                                                  ('cat_enc',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  ['de_natureza_juridica',
                                                   'sg_uf',
                                                   'natureza_juridica_macro',
                                                   'de_ramo', 'setor',
                                                   'idade_emp_cat',
                                                   'dt_situacao', 'fl_rm',
               

In [50]:
pipe.named_steps['model'].kneighbors(\
    data_transformer.transform(X.sample(10)), 3
)

(array([[5.55111512e-16, 2.05465055e-01, 2.77860288e-01],
        [1.11022302e-16, 1.25952919e-01, 1.43742798e-01],
        [0.00000000e+00, 9.07825832e-02, 1.36173875e-01],
        [0.00000000e+00, 3.41571150e-02, 6.78092893e-02],
        [3.33066907e-16, 4.27934637e-02, 4.27934637e-02],
        [0.00000000e+00, 9.35976639e-02, 9.35976639e-02],
        [0.00000000e+00, 1.29653282e-01, 1.60097573e-01],
        [0.00000000e+00, 4.12081388e-02, 7.60486159e-02],
        [1.11022302e-16, 5.09885017e-02, 7.30535107e-02],
        [0.00000000e+00, 1.68929765e-01, 1.98128266e-01]]),
 array([[7139, 3839, 6369],
        [7284, 3124,  365],
        [6829, 7392,  425],
        [3683, 9482, 2002],
        [4004, 3375, 3656],
        [7045,  591, 6892],
        [6120, 2267, 4092],
        [ 682, 2948, 9191],
        [5393, 7446, 1882],
        [1712, 8489, 7703]]))

# Testes

In [40]:
sample_ids = X.sample(10)['id']

In [45]:
sample_ids

283261    a4fbab5d8b99aeba21f0bd3bff115f2f1ca055453f165d...
386015    9f48dff3e7decc17dcd9204f2714a26fd5606fbaedf6ca...
68900     362736b7b3f30cd91f8d86d9e06f938b7a4b67e6855ed8...
348431    cd4bdbccebd9109c2a9982712a0fe512afbc257d81c8b7...
332473    e59198d7ad05dc980859a7609c92ed15e5e031527fafc2...
131132    e01d31801de9b7fb7ddbeb069f530c6b4eeeb77d4c5cf5...
300668    250427e68906aeb7fa06a06efbc7b8d0063721a53cae77...
255316    c897114b61c3f0f6a690d1a64d9d14558ef3f5cfb7d474...
357216    afacf5c5c138242a989bbe7e0727757117ead7f97ac77c...
394562    acf42bc24a8e066632c1b10b3f21b74d9a75b682e3248a...
Name: id, dtype: object

In [82]:
def recommendations(ids: pd.Series, n: int):
    '''
    Extrai recomendações do modelo.
    '''
    data = X.loc[ids.index, :]
    
    distances, indexes = pipe.named_steps['model'].kneighbors(\
        data_transformer.transform(data), n
    )
    
    df = X.iloc[idx.flatten().tolist(), :]
    
    return distances, indexes, df

In [83]:
ds, idx, _ = recommendations(sample_ids, 4)

In [84]:
_

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
283261,a4fbab5d8b99aeba21f0bd3bff115f2f1ca055453f165d...,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,SERVIÇO,13.813699,10 a 15,0,0,0,0,0,2005-01-07,0,0,0,NAO,MANUTE

# Pickling

In [85]:
# exporta o modelo treinado
joblib.dump(pipe, '../model.joblib')

['model.joblib']